**Project Overview-** The goal of this project is to analyze market data of 18 different ETFs in order to create a well balanced porfolio that meets the needs of a 59-year-old investor. This investor is nearing retirement, but does not have enough invested into a retirement fund to support their lifestyle when they eventually retire. Due to this, they are hoping to develop a portfolio that is aggressive enough to help them catch up, while still being stable enough to hedge the risk of losing all their current savings. Our investor is risk-moderate and wants to invest in funds rather than individual equities.

**Goal Return-** The investor of this portfolio is hoping to realize a return of about 1.5x the market, or about 10.5% based on a typical market return of 7%

**Goal Beta (risk)-** The investor of this portfolio is willing to invest with a beta of 1.5. They do not want excessive risk but are willing to take on more than a traditional late stage retirement fund in order to open up the opportunty for greater return. 

**Goal Allocation-** The investor, in hopes to atain their desired returns would like an investment mix with 65% invested in equities and 35% invested in stable funds.

**Questions to Answer**

1. How wide is the range of returns found in the data set?
2. Which fund types provide the highest return? Which fund types provide the best dividends?
3. What was the overall performance of all these funds over the past five years? What about the individual fund types? 
4. How much return can I expect to realize should I invest in these funds? (What is the average APR of return for each fund?)
5. How risky is each individual fund? (St.Dev)
6. How do the returns on these funds compare to the overall market? 
7. How does the riskiness of these funds compare to the overall market? (Beta)
8. How do the expected returns of these individual funds compare to their actual returns? (CAPM)
9. What is the ideal mix/allocation of funds to match the goals of this portfolio?

**Step 1: Import Pandas / Numpy / Plotly Libraries**

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import linregress

**Step 2: Load Data from GitHub and Explore**

In [ ]:
df = pd.read_csv('https://github.com/zachcep/Portfolio-Analysis/raw/main/PortfolioAnalysisData.csv',encoding='iso-8859-1')
df

In [3]:
df.shape

(1140, 6)

In [ ]:
df.info

In [ ]:
df.head(100)

In [ ]:
df.tail(100)

In [ ]:
df.sample(100)

**Step 3: Adjusting Datetime Columns**

In [ ]:
df['Date'] = pd.to_datetime(df['Date'],format='%m/%d/%Y')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df2 = df[['Month','Year','Fund','Type','Open','Close','Dividend']]
df2['Dividend'] = df2['Dividend'].fillna(0)
df2

**Step 4: Aggregate Monthly Returns**

In [ ]:
df2['Monthly Returns'] = ((df2['Close'] - df2['Open'] + df2['Dividend'])/df2['Open'])
df2

In [10]:
returns_fig = px.box(data_frame=df2,x='Monthly Returns',orientation='h',title='Monthly Returns')
returns_fig.show()

In [11]:
returns_fig2 = px.histogram(data_frame=df2,x='Monthly Returns',nbins=6,title='Monthly Returns Spread')
returns_fig2.show()

**Step 5: Analyze Data by Fund Type**

In [12]:
df_type = df2.groupby(by='Type',as_index=False).agg({'Monthly Returns':'mean','Dividend':'mean'})
df_type.rename(columns={'Monthly Returns':'Avg Monthly Returns','Dividend':'Avg Monthly Dividend'},inplace=True)
df_type.sort_values(by='Avg Monthly Returns',ascending=False,inplace=True)
df_type

,Type,Avg Monthly Returns,Avg Monthly Dividend
1,Growth,0.018530,0.147133
4,Market,0.011657,0.000000
5,Mid Cap,0.010887,0.103500
2,Income,0.010518,0.141272
3,Large Cap,0.010495,0.165056
6,Small Cap,0.009619,0.165989
0,Bonds,0.003471,0.081572


In [13]:
df_type['Avg Yearly Returns'] = df_type['Avg Monthly Returns']*12
df_type['Avg Yearly Dividend'] = df_type['Avg Monthly Dividend']*12
df_type_yearly = df_type.drop(columns=['Avg Monthly Returns','Avg Monthly Dividend'])
df_type_yearly

,Type,Avg Yearly Returns,Avg Yearly Dividend
1,Growth,0.222363,1.765600
4,Market,0.139879,0.000000
5,Mid Cap,0.130643,1.242000
2,Income,0.126215,1.695267
3,Large Cap,0.125934,1.980667
6,Small Cap,0.115429,1.991867
0,Bonds,0.041651,0.978867


In [14]:
type_fig = px.bar(data_frame=df_type_yearly,x='Type',y='Avg Yearly Returns',title='Returns by Fund Type',template='plotly_dark')
type_fig.show()

In [15]:
type_fig2 = px.bar(data_frame=df_type_yearly,x='Type',y='Avg Yearly Dividend',title='Dividend by Fund Type',template='plotly_dark')
type_fig2.show()

**Step 6: Analyze Data by Year**

In [16]:
df_by_year = df2.groupby(by='Year',as_index=False).agg({'Monthly Returns':'mean','Dividend':'mean'})
df_by_year.rename(columns={'Monthly Returns':'Avg Monthly Returns','Dividend':'Avg Monthly Dividend'},inplace=True)
df_by_year.sort_values(by='Year',ascending=False,inplace=True)
df_by_year

,Year,Avg Monthly Returns,Avg Monthly Dividend
5,2021,0.012241,0.165232
4,2020,0.017435,0.134509
3,2019,0.018692,0.135610
2,2018,-0.005832,0.131947
1,2017,0.012816,0.113592
0,2016,0.008577,0.086820


In [17]:
df_by_year['Avg Yearly Returns'] = df_by_year['Avg Monthly Returns']*12
df_by_year['Avg Yearly Dividend'] = df_by_year['Avg Monthly Dividend']*12
df_by_year_yearly = df_by_year.drop(columns=['Avg Monthly Returns','Avg Monthly Dividend'])
df_by_year_yearly

,Year,Avg Yearly Returns,Avg Yearly Dividend
5,2021,0.146897,1.982779
4,2020,0.209217,1.614105
3,2019,0.224300,1.627316
2,2018,-0.069986,1.583368
1,2017,0.153795,1.363105
0,2016,0.102929,1.041835


In [18]:
by_year_fig = px.line(data_frame=df_by_year_yearly,x='Year',y='Avg Yearly Returns',title='Returns Over Past 5-Years',template='plotly_dark')
by_year_fig.show()

In [19]:
by_year_fig2 = px.line(data_frame=df_by_year_yearly,x='Year',y='Avg Yearly Dividend',title='Dividend Over Past 5-Years',template='plotly_dark')
by_year_fig2.show()

**Step 7: Graph Data by Both Year and Fund**

In [20]:
df_graph = df2.groupby(by=['Year','Type'],as_index=False).agg({'Monthly Returns':'mean','Dividend':'mean'})
df_graph.rename(columns={'Monthly Returns':'Avg Monthly Returns','Dividend':'Avg Monthly Dividend'},inplace=True)
df_graph.sort_values(by='Year',ascending=False,inplace=True)
df_graph['Avg Yearly Returns'] = df_graph['Avg Monthly Returns']*12
df_graph['Avg Yearly Dividend'] = df_graph['Avg Monthly Dividend']*12
df_graph2 = df_graph.drop(columns=['Avg Monthly Returns','Avg Monthly Dividend'])

In [21]:
by_both_fig = px.line(data_frame=df_graph2,x='Year',y='Avg Yearly Returns',color='Type',title='Returns Over Past 5-Years by Fund Type',template='plotly_dark')
by_both_fig.show()

In [22]:
by_both_fig2 = px.line(data_frame=df_graph,x='Year',y='Avg Yearly Dividend',color='Type',title='Dividend Over Past 5-Years by Fund Type',template='plotly_dark')
by_both_fig2.show()

**Step 8: Analyze Individual Funds by Type**

In [23]:
df2.Type.unique()

array(['Income', 'Bonds', 'Growth', 'Large Cap', 'Mid Cap', 'Small Cap',
       'Market'], dtype=object)

In [24]:
df_income = df2[df2['Type'] == 'Income']
df_bonds = df2[df2['Type'] == 'Bonds']
df_large_cap = df2[df2['Type'] == 'Large Cap']
df_mid_cap = df2[df2['Type'] == 'Mid Cap']
df_small_cap = df2[df2['Type'] == 'Mid Cap']
df_market = df2[df2['Type'] == 'Market']

***Income Funds***

In [ ]:
df_income['St.Dev of Returns'] = df_income['Monthly Returns']

In [26]:
df_income2 = df_income.groupby(by='Fund',as_index=False).agg({'Monthly Returns':'mean','Dividend':'mean','St.Dev of Returns':'std'})
df_income2.rename(columns={'Monthly Returns':'Avg Monthly Returns','Dividend':'Avg Monthly Dividend'},inplace=True)
df_income2

,Fund,Avg Monthly Returns,Avg Monthly Dividend,St.Dev of Returns
0,Schuab U.S. Divided Equity (SCHD),0.012895,0.133383,0.047604
1,Vanguard Dividend Appreciation Index Fund (VIG),0.011842,0.155717,0.042099
2,iShares International Select Dividend (IDV),0.006816,0.134717,0.054880


In [27]:
df_income2['APR'] = df_income2['Avg Monthly Returns']*12
df_income2['Avg Yearly Dividend'] = df_income2['Avg Monthly Dividend']*12
df_income3 = df_income2.drop(columns=['Avg Monthly Returns','Avg Monthly Dividend'])

In [28]:
df_income4 = df_income3.drop(columns='Avg Yearly Dividend')
df_income4

,Fund,St.Dev of Returns,APR
0,Schuab U.S. Divided Equity (SCHD),0.047604,0.154745
1,Vanguard Dividend Appreciation Index Fund (VIG),0.042099,0.142106
2,iShares International Select Dividend (IDV),0.054880,0.081792


Monthly Returns for Beta Calculations

In [43]:
SCHD = df_income[df_income['Fund'] == 'Schuab U.S. Divided Equity (SCHD)']['Monthly Returns']
VIG = df_income[df_income['Fund'] == 'Vanguard Dividend Appreciation Index Fund (VIG)']['Monthly Returns']
IDV = df_income[df_income['Fund'] == 'iShares International Select Dividend (IDV)']['Monthly Returns']
Market = df_market['Monthly Returns']

In [44]:
beta = linregress(VIG,Market)
beta[0]

1.061147119040196

In [31]:
df_income_sp = df2[(df2['Type'] == 'Income') | (df2['Type'] == 'Market')]
df_bonds_sp = df2[(df2['Type'] == 'Bonds') | (df2['Type'] == 'Market')]
df_large_cap_sp = df2[(df2['Type'] == 'Large Cap') | (df2['Type'] == 'Market')]
df_mid_cap_sp = df2[(df2['Type'] == 'Mid Cap') | (df2['Type'] == 'Market')]
df_small_cap_sp = df2[(df2['Type'] == 'Mid Cap') | (df2['Type'] == 'Market')]
df_market_sp = df2[(df2['Type'] == 'Market') | (df2['Type'] == 'Market')]